# Foreign exchange rates
#### Using Bank of Canada's Valet API

In [1]:
import requests
import pandas as pd

In [2]:
# get exchange rate data using valet api
url = "https://www.bankofcanada.ca/valet/observations/group/FX_RATES_DAILY/json"
response = requests.get(url)
data = response.json()

In [10]:
# set currencies (e.g., 'CAD', 'EUR', 'MXN', 'PHP', 'USD')
foreign_currency = 'EUR'  # set foreign currency 
target_currency = 'CAD'  # keep target currency as CAD

# extract exchange rates
rates = data['observations']

# construct key for desired currency pair
exchange_rate_key = f"FX{foreign_currency}{target_currency}"

# extract rates for currency pair
exchange_rates = [
    {
     "date": obs["d"], 
     "rate": float(obs[exchange_rate_key]["v"])
    }
    for obs in rates if exchange_rate_key in obs
                 ]
df = pd.DataFrame(exchange_rates)
df.rename(columns={'rate': f'{foreign_currency} to CAD'}, inplace=True)

# add column for CAD to foreign currency
df["CAD to " + foreign_currency] = 1 / df[f'{foreign_currency} to CAD']

# display results
print('Foreign exchange rate:', foreign_currency, 'to Canadian Dollars', '\n\n')
df.tail()


Foreign exchange rate: EUR to Canadian Dollars 




,date,EUR to CAD,CAD to EUR
1997,2025-01-06,1.4909,0.670736
1998,2025-01-07,1.4871,0.672450
1999,2025-01-08,1.4829,0.674354
2000,2025-01-09,1.4829,0.674354
2001,2025-01-10,1.4779,0.676636
